In [1]:
import pandas as pd

# Load the s&p500 csv file to read the csv names
sp500_components = pd.read_csv('sp500.csv')

In [2]:
import numpy as np

# Create a matrix of 4482 by ~504 by 4, I only need 4 columns per stock, per day, for 4482 days of data or 18 years.
days = 4482   
components = len(sp500_components)
columns = 4    # I'm only using High, Low, Open and Close columns 

prepared_data_matrix = np.zeros((days,components,columns))




In [3]:
def add_to_data_matrix(csv_file, index):
    
    csv_data = pd.read_csv(csv_file)
    
    date = (len(prepared_data_matrix) - 1)
    data_size = (len(csv_data) -1)
    
    if date <= data_size:
        counter = date   # 4528 spots counting from the 0th index
    else:
        counter = data_size # Some number less than 4528
    
    print('counter =', counter)
    
    while counter >= 0:
        
        prepared_data_matrix[date][index][0] = csv_data['Open'][counter]
        prepared_data_matrix[date][index][1] = csv_data['High'][counter]
        prepared_data_matrix[date][index][2] = csv_data['Low'][counter]
        prepared_data_matrix[date][index][3] = csv_data['Close'][counter]
        date -= 1
        counter -= 1
        
    print('Finished adding', csv_file, 'data to the prepared_data_matrix!')


In [4]:
import os.path   # To check whether a file exist

for index in range (len(sp500_components) - 1):
    
    stock_symbol = str(sp500_components['Symbol'][index])    # Get the stock symbol
    csv_file = ''.join([stock_symbol, '.csv'])    # Create the filename to read from the local directory 
    
    # Read files found in the local directory
    if os.path.isfile(csv_file):
        print(index, csv_file)
        add_to_data_matrix(csv_file, index)
    else:
        print("File not found!")
        
print('Done creating prepared_data_matrix!')

0 A.csv
counter = 4481
Finished adding A.csv data to the prepared_data_matrix!
1 AAL.csv
counter = 3144
Finished adding AAL.csv data to the prepared_data_matrix!
2 AAP.csv
counter = 4107
Finished adding AAP.csv data to the prepared_data_matrix!
3 AAPL.csv
counter = 4481
Finished adding AAPL.csv data to the prepared_data_matrix!
4 ABBV.csv
counter = 1315
Finished adding ABBV.csv data to the prepared_data_matrix!
5 ABC.csv
counter = 4481
Finished adding ABC.csv data to the prepared_data_matrix!
6 ABMD.csv
counter = 4481
Finished adding ABMD.csv data to the prepared_data_matrix!
7 ABT.csv
counter = 4481
Finished adding ABT.csv data to the prepared_data_matrix!
8 ACN.csv
counter = 4196
Finished adding ACN.csv data to the prepared_data_matrix!
9 ADBE.csv
counter = 4481
Finished adding ADBE.csv data to the prepared_data_matrix!
10 ADI.csv
counter = 4481
Finished adding ADI.csv data to the prepared_data_matrix!
11 ADM.csv
counter = 4481
Finished adding ADM.csv data to the prepared_data_matrix

counter = 4481
Finished adding CI.csv data to the prepared_data_matrix!
100 CINF.csv
counter = 4481
Finished adding CINF.csv data to the prepared_data_matrix!
101 CL.csv
counter = 4481
Finished adding CL.csv data to the prepared_data_matrix!
102 CLX.csv
counter = 4481
Finished adding CLX.csv data to the prepared_data_matrix!
103 CMA.csv
counter = 4481
Finished adding CMA.csv data to the prepared_data_matrix!
104 CMCSA.csv
counter = 4481
Finished adding CMCSA.csv data to the prepared_data_matrix!
105 CME.csv
counter = 3850
Finished adding CME.csv data to the prepared_data_matrix!
106 CMG.csv
counter = 3061
Finished adding CMG.csv data to the prepared_data_matrix!
107 CMI.csv
counter = 4481
Finished adding CMI.csv data to the prepared_data_matrix!
108 CMS.csv
counter = 4481
Finished adding CMS.csv data to the prepared_data_matrix!
109 CNC.csv
counter = 4097
Finished adding CNC.csv data to the prepared_data_matrix!
110 CNP.csv
counter = 4481
Finished adding CNP.csv data to the prepared_da

Finished adding FOXA.csv data to the prepared_data_matrix!
197 FRT.csv
counter = 4481
Finished adding FRT.csv data to the prepared_data_matrix!
198 FTI.csv
counter = 4219
Finished adding FTI.csv data to the prepared_data_matrix!
199 FTV.csv
counter = 434
Finished adding FTV.csv data to the prepared_data_matrix!
200 GD.csv
counter = 4481
Finished adding GD.csv data to the prepared_data_matrix!
201 GE.csv
counter = 4481
Finished adding GE.csv data to the prepared_data_matrix!
202 GGP.csv
counter = 4481
Finished adding GGP.csv data to the prepared_data_matrix!
203 GILD.csv
counter = 4481
Finished adding GILD.csv data to the prepared_data_matrix!
204 GIS.csv
counter = 4481
Finished adding GIS.csv data to the prepared_data_matrix!
205 GLW.csv
counter = 4481
Finished adding GLW.csv data to the prepared_data_matrix!
206 GM.csv
counter = 1848
Finished adding GM.csv data to the prepared_data_matrix!
207 GOOG.csv
counter = 1006
Finished adding GOOG.csv data to the prepared_data_matrix!
208 GOOGL

Finished adding LUV.csv data to the prepared_data_matrix!
295 LYB.csv
counter = 1991
Finished adding LYB.csv data to the prepared_data_matrix!
296 M.csv
counter = 4481
Finished adding M.csv data to the prepared_data_matrix!
297 MA.csv
counter = 2977
Finished adding MA.csv data to the prepared_data_matrix!
298 MAA.csv
counter = 4481
Finished adding MAA.csv data to the prepared_data_matrix!
299 MAC.csv
counter = 4481
Finished adding MAC.csv data to the prepared_data_matrix!
300 MAR.csv
counter = 4481
Finished adding MAR.csv data to the prepared_data_matrix!
301 MAS.csv
counter = 4481
Finished adding MAS.csv data to the prepared_data_matrix!
302 MAT.csv
counter = 4481
Finished adding MAT.csv data to the prepared_data_matrix!
303 MCD.csv
counter = 4481
Finished adding MCD.csv data to the prepared_data_matrix!
304 MCHP.csv
counter = 4481
Finished adding MCHP.csv data to the prepared_data_matrix!
305 MCK.csv
counter = 4481
Finished adding MCK.csv data to the prepared_data_matrix!
306 MCO.csv

Finished adding RE.csv data to the prepared_data_matrix!
392 REG.csv
counter = 4481
Finished adding REG.csv data to the prepared_data_matrix!
393 REGN.csv
counter = 4481
Finished adding REGN.csv data to the prepared_data_matrix!
394 RF.csv
counter = 4481
Finished adding RF.csv data to the prepared_data_matrix!
395 RHI.csv
counter = 4481
Finished adding RHI.csv data to the prepared_data_matrix!
396 RHT.csv
counter = 4481
Finished adding RHT.csv data to the prepared_data_matrix!
397 RJF.csv
counter = 4481
Finished adding RJF.csv data to the prepared_data_matrix!
398 RL.csv
counter = 4481
Finished adding RL.csv data to the prepared_data_matrix!
399 RMD.csv
counter = 4481
Finished adding RMD.csv data to the prepared_data_matrix!
400 ROK.csv
counter = 4481
Finished adding ROK.csv data to the prepared_data_matrix!
401 ROP.csv
counter = 4481
Finished adding ROP.csv data to the prepared_data_matrix!
402 ROST.csv
counter = 4481
Finished adding ROST.csv data to the prepared_data_matrix!
403 RSG.

Finished adding WU.csv data to the prepared_data_matrix!
490 WY.csv
counter = 4481
Finished adding WY.csv data to the prepared_data_matrix!
491 WYNN.csv
counter = 3879
Finished adding WYNN.csv data to the prepared_data_matrix!
492 XEC.csv
counter = 3901
Finished adding XEC.csv data to the prepared_data_matrix!
493 XEL.csv
counter = 4481
Finished adding XEL.csv data to the prepared_data_matrix!
494 XL.csv
counter = 4481
Finished adding XL.csv data to the prepared_data_matrix!
495 XLNX.csv
counter = 4481
Finished adding XLNX.csv data to the prepared_data_matrix!
496 XOM.csv
counter = 4481
Finished adding XOM.csv data to the prepared_data_matrix!
497 XRAY.csv
counter = 4481
Finished adding XRAY.csv data to the prepared_data_matrix!
498 XRX.csv
counter = 4481
Finished adding XRX.csv data to the prepared_data_matrix!
499 XYL.csv
counter = 1621
Finished adding XYL.csv data to the prepared_data_matrix!
500 YUM.csv
counter = 4481
Finished adding YUM.csv data to the prepared_data_matrix!
501 ZB

In [5]:
# Load the SPY also known as SPYDR
spydr_data = pd.read_csv('SPY.csv')
y = spydr_data[['Open', 'High', 'Low', 'Close']].values

In [6]:
y.shape

(4482, 4)

In [7]:
prepared_data_matrix.shape

(4482, 505, 4)

In [8]:
# Create 2d matrix from 3d.

X = prepared_data_matrix.reshape(days * components, columns)
#X = prepared_data_matrix

In [11]:
X.tofile('X.tsv', sep="\t", format="%s")

In [ ]:
# Plots
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
# Clustering
from sklearn.manifold import TSNE
from sklearn import cluster
# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool

In [ ]:
%matplotlib inline
output_notebook()

In [ ]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
#TSNE
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')

In [ ]:
X_tsne = tsne_model.fit_transform(X)

In [ ]:
cl = cluster.AgglomerativeClustering(10)
cl.fit(X_tsne)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
cmap = plt.cm.get_cmap('jet')
plt.scatter(X_tsne[:len(y),0], X_tsne[:len(y),1], 
            alpha=0.5, c=y, cmap=cmap, s=20)
plt.colorbar()
plt.subplot(1,2,2)
plt.scatter(X_tsne[len(y):,0], X_tsne[len(y):,1], 
            alpha=0.4, c=cl.labels_[len(y):], marker='s', s=20)

In [ ]:
plt.show()